## Chapter 5 : Hands-On usage of WorkLoop, toolbox::Event, Event dispatch/callback, Exception handling

Besides the xdata::Event we used in the last chapter. toolbox::Event dispatch and callback can be seens as another way to hook into the main program flow. One can create its own type of toolbox::Event. WorkLoop is the xdaq way to spawn a separate thread to do some work for you. 


### Example (part 1)

 SmartDataSource is a xdaq application who is capable of self-regulating the queue population. The queue is filled with a workloop "pushing" and popped with a workloop "popping" where the pushing rate is twice of the popping rate initially. A **"QueueAlmostFull"** event is fired by the application when the queue size reaches 80% of its max allowed size. Reacting to this event, the queue popping rate is increased by a factor of 2. A **"QueueAlmostEmpty"** event is fired when the queue size reaches 20% of its max size. Reacting to this event, the queue popping rate is decreased by a factor of 2. It is the dynamic change in the popping rate that keeps the queue size in balance. 

Export again from svn example directory and set up your working environment as described in Chapter 2.

Choose either modify the configuration file by hand $workdir/daq/bril/mypackage/xml/SmartDataSource.xml 

*or* 

generate it from template:

```
cd $workdir/daq/bril/scripts

./generatexml.sh mypackage/SmartDataSource

```

Build mypackage library:


```
make install

```

Go to where the configuration xml resides and launch the executive from there:

```
/opt/xdaq/bin/xdaq.exe -p $your_context_port -e /opt/xdaq/etc/default.profile -c `pwd`/SmartDataSource.xml
```

Observe the change in the queue size and occupancy from logging on the screen.

From the source code find the parts responsible for each action and reaction. Learn how to define and submit a workloop. 

How many threads are in the play?

Can the "pushing" workloop be a timer instead? 

Can the "popping" workloop be a timer instead?

Why we must "return true" from the workloop functions in this application? 

( This part of the example does not require using a web browser. )


**Note:**

You should always use usleep in the **recurring** type of workloops. You should try to usleep as long as your situation permits. Because a very fast recurring workloop will generate very high CPU load ( and burn your machine ).


**A word on thread-safty:**

This example works without any explicit usage of semaphores because toolbox::squeue is a thread-safe object. When a structured and non-thread-safe object can to be modified and accessed by multiple threads, you must take into account the thread-saftiness in your design. Ask yourself questions like: Is it possible that the object is partially correct when it is read? Is it possible that I'm in a dead-lock situation?

xdaq framework provides some utilities to manage thread-safty issues, frequently used are toolbox::BSem and toolbox::task::Guard. 

### Example (part 2)

The SmartDataSource itself can maintain the balance so that the queue is never overflown or depleted. Its hyperdaq page has two buttons that can disturbe such balance. 

Open the hyperdaq page of this application. Click on the "Push" or "Pop" buttons quickly to overflow or deplete the queue. 

In this application, queue overflow and empty are exceptional conditions (though empty queue in real world is not so exceptional). The difference is that in the case of queue overflow, it is already an exception from toolbox::squeue, and we demonstrate how to handle exceptions from others. In the case of empty queue, we want to raise and handle an exception of our own.  

In the example, we demonstrate the usage several xdaq macros to raise or rethrow an exception. 

Identify where and what they are. 

Switch (comment/uncomment) between different exception handlings to see what happens. 

A un-caught exception causes the program to crash. The flow of the program will be halted. Sometimes this behaviour is exactly what you wanted, most of the time it is not the wanted behavior especially for an always-running service.

**notifyQualified** method allows throwing an exception without halting the program while the exception is registered in the error database and can be visualized or analyzed later.

To understand the meaning of the last sentence:

Edit SmartDataSource.cc modify the exception handling to use notifyQualified.

Open a new tab in web browser, and go to xdaq smarthub page: https://xdaq.web.cern.ch/xdaq/xmas/14/smarthub , bril , sentinel:spotlightocci, then Hotspot Flex Interface. Keep this page open

First launch the executive outside the bril zone with -e , and generate queue overflow condition with the "Push" button, observe what happens in the Hotspot page.

Then launch again the executive but inside the bril zone with -z bril, generate the same exceptional condition as before, you should see the errors are collected by the sentinel service and appear in the Hotspot page.

(Smarthub pages use flash, you might have display problem if the flash installation in your browser is not good. In that case, find out the direct hotspot's application URL and enter from there.) 

Note: 

In general, catch an exception but do nothing about it is BAD. It is called "swallow" an exception: none of the downstream classes/methods can catch it ever again while the exceptional condition is not handled by you either. 


### Excercise

We inherit from different base classes in this example, what they are and what the purpose of each?

Change return true to return false in the workloop functions to see what happens.


(Chanllenging) Rewrite the program with std::dequeue, identify potential multithreading issues and use toolbox::BSem and toolbox::task::Guard to protect the queue.